#### Build a model to predict king county house sale prices

To build a linear regression model to predict the house sale prices and evaluate models according to MAE (median absolute error) on houses with sale prices between the 5th and 95th percentiles.

##### Data Preparation

In [ ]:
import pandas as pd
import numpy as np 
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt    
%matplotlib inline


In [ ]:
sales_df = pd.read_csv('raw_data/EXTR_RPSale.csv')
resbldg_df = pd.read_csv('raw_data/EXTR_ResBldg.txt', sep='\t')

##### Load and clean data

In [ ]:
sales_df.head()
resbldg_df.head()

In [ ]:
sales_df.info()
resbldg_df.info()

In [ ]:
sales_df.describe().T

In [ ]:
sales_df.duplicated().sum() #0 duplicated rows
resbldg_df.duplicated().sum()    #0 duplicated rows

In [ ]:
sales_df.isna().sum()    #no NAs
resbldg_df.isna().sum()     #has NAs

In [ ]:
sales_df1 = sales_df.loc[:, (sales_df != 0).any(axis=0)]   #No rows with all zeros
print(sales_df.shape)
print(sales_df1.shape)

In [ ]:
resbldg_df1 = resbldg_df.loc[:, (resbldg_df != 0).any(axis=0)]   #No rows with all zeros
print(resbldg_df.shape)
print(resbldg_df1.shape)

In [ ]:
resbldg_df.columns    #space in column headers

In [ ]:
resbldg_df2 = resbldg_df.rename(columns={'Major ': 'Major', 'SqFtDeck  ': 'SqFtDeck'})   #renaming column header

In [ ]:
#choosing initial features
sales_df2 = sales_df1[['Major', 'Minor', 'SalePrice']]     
resbldg_df3 = resbldg_df2[['Major', 'Minor', 'SqFtTotLiving', 'Bedrooms', 'SqFtGarageAttached',   
                   'Condition', 'SqFtDeck','BathFullCount', 'ZipCode','HeatSystem', 'YrBuilt']] 

resbldg_df3.head()

In [ ]:
sales_df2['Major'] = pd.to_numeric(sales_df2['Major'], errors='coerce')    #objects into float
sales_df2['Minor'] = pd.to_numeric(sales_df2['Minor'], errors='coerce')
resbldg_df3['Major'] = pd.to_numeric(resbldg_df3['Major'], errors='coerce')

sales_data = pd.merge(sales_df2, resbldg_df3, on=['Major', 'Minor'])       #merge datasets
sales_data.head()

Histogram to see the sale price distribution.

In [ ]:
#For initial visualization, droppping the top and lowest 10% of sale prices 
lowest_10percent = np.percentile(sales_data['SalePrice'], 10)
highest_90percent = np.percentile(sales_data['SalePrice'], 90)

sales_data1 = sales_data[sales_data['SalePrice'] > lowest_10percent]
sales_data2 = sales_data1[sales_data1['SalePrice'] < highest_90percent] 

In [ ]:
plt.hist(sales_data2['SalePrice'], bins=30000)
plt.show()

#### Histogram above shows that sale prices are not normally distributed. Therefore, we log transformed and regenerated histogram on all SalePrices. 

In [ ]:
#Histogram shows log transformed prices are normally distributed.
fig, ax = plt.subplots()
ax.hist(np.log(sales_data[sales_data['SalePrice']> 0]['SalePrice']), bins=50);

##### Feature Engineering

In [ ]:
# Handling the null values
nulls = pd.DataFrame(sales_data.isnull().sum().sort_values(ascending=False))    
nulls.columns = ['Null Count']
nulls.index.name = 'Feature'
print(nulls)

In [ ]:
#remove nulls in ZipCode
sales_data = sales_data.dropna(subset = ['ZipCode'])                     
nulls1 = pd.DataFrame(sales_data1.isnull().sum().sort_values(ascending=False))
print(nulls1)

sales_data1.shape                        

In [ ]:
#Keep only the positive values for sale price
sales_data = sales_data[sales_data['SalePrice'] > 0]
sales_data['SalePrice'].min()
sales_data.shape

In [ ]:
# Adding LogSalePrice values as a new column to build the model on log sale prices as our target   
# instead of raw sale prices.
sales_data['LogSalePrice']=np.log(sales_data['SalePrice'])
sales_data.head().T

Log of SalePrice is normally distributed so we kept 95% of our target values by limiting to the target values larger than mean minus two standard deviation and smaller than mean plus two standard deviation. 

In [ ]:
# keeping the target values above mean minus two standard deviation and below mean plus two standard deviation
print(sales_data['LogSalePrice'].mean())
print(sales_data['LogSalePrice'].std())

mean_minus_twostdv = sales_data['LogSalePrice'].mean() - sales_data['LogSalePrice'].std()*2
mean_plus_twostdv = sales_data['LogSalePrice'].mean() + sales_data['LogSalePrice'].std()*2
print(mean_minus_twostdv)
print(mean_plus_twostdv)

In [ ]:
sales_data = sales_data[(sales_data['LogSalePrice'] < mean_plus_twostdv) & 
                         (mean_minus_twostdv < sales_data['LogSalePrice'])]
sales_data.head()
sales_data['LogSalePrice'].describe()

In [ ]:
#keep only the first 5 digits of ZipCode.
def is_integer(x):
   try:
       _ = int(x)
   except ValueError:
       return False
   return True

stringed_zips = sales_data.loc[sales_data['ZipCode'].apply(is_integer) == False, 'ZipCode']
stringed_zips.head()

In [ ]:
def zipcode_short(x):
    return str(x)[:5]

sales_data['New_zip'] = sales_data['ZipCode'].map(zipcode_short)
sales_data.head().T
sales_data.shape

In [ ]:
#remove "WA", space, and "A","B" as ZipCodes
sales_data.drop(sales_data[ sales_data['New_zip'] == "WA" ].index , inplace=True)
sales_data.drop(sales_data[ sales_data['New_zip'] == " " ].index , inplace=True)
sales_data.drop(sales_data[ sales_data['New_zip'] == "B" ].index , inplace=True)
sales_data.drop(sales_data[ sales_data['New_zip'] == "A" ].index , inplace=True)

stringed_zips = sales_data.loc[sales_data['New_zip'].apply(is_integer) == False, 'New_zip']
stringed_zips

In [ ]:
#Initial correlation top 5.
numeric_feature = sales_data.select_dtypes(include=[np.number])
corr = numeric_feature.corr()

print(abs(corr['LogSalePrice']).sort_values(ascending=False))

In [ ]:
#concatenate Major and Minor into PIN
sales_data['Major'] = sales_data.Major.astype(int)
sales_data['Minor'] = sales_data.Minor.astype(int)

In [ ]:
sales_data['Pin'] = sales_data.Major.astype(str) + sales_data.Minor.astype(str)
sales_data.shape

In [ ]:
#Drop duplicated rows
sales_data = sales_data.drop_duplicates()
sales_data.shape

In [ ]:
#Choose the top correlated features
sales_data.columns
sales_data = sales_data[['LogSalePrice', 'SqFtTotLiving', 'Bedrooms', 'SqFtGarageAttached', 'SqFtDeck', 
                          'BathFullCount', 'YrBuilt', 'New_zip', 'Pin']].copy()
sales_data.head()

##### Visualization

In [ ]:
#Heatmap of features
def correlation_heatmap(df1):
   _, ax = plt.subplots(figsize = (15, 10))
   colormap= sns.diverging_palette(220, 10, as_cmap = True)
   sns.heatmap(df1.corr(), annot=True, cmap = colormap)

correlation_heatmap(sales_data)

In [ ]:
#See the distribution of house sales based on ZipCode
plt.xlabel('houses sold')
plt.ylabel('counts')
plt.hist(sales_data['New_zip'].value_counts())

In [ ]:
zipcode_counts = sales_data['New_zip'].value_counts().to_dict()

In [ ]:
sales_data['Zip_count'] = sales_data['New_zip'].apply(lambda z: zipcode_counts[z])

In [ ]:
#Keep the ZipCodes with over 15000 house sales
sales_data_over_15k = sales_data[sales_data['Zip_count'] > 15000]
sales_data_over_15k.head()

In [ ]:
Zip_by_price = sales_data_over_15k.groupby('New_zip')['LogSalePrice'].mean().sort_values()
Zip_by_price

In [ ]:
zcodes = list(Zip_by_price.index)
for zc in zcodes:
    sales_data_over_15k[zc] = (sales_data_over_15k['New_zip'] == zc).astype(int)

In [ ]:
sales_data_over_15k.head()

#### Initial correlation of zipcodes with logsalePrice.

In [ ]:
numeric_feature = sales_data_over_15k.select_dtypes(include=[np.number])
corr = numeric_feature.corr()

print(abs(corr['LogSalePrice']).sort_values(ascending = False))

#### Build a multiple linear regression with top 7 features.

In [ ]:
Y = sales_data_over_15k['LogSalePrice']
important_feat = [ele for ele in sales_data_over_15k.columns if ele in {
    'SqFtTotLiving', 'BathFullCount', 'Bedrooms', 'SqFtGarageAttached','SqFtDeck','YrBuilt','98006'}] 
X = sales_data_over_15k[important_feat]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state = 1000, test_size=.3)
lr = linear_model.LinearRegression()

In [ ]:
model = lr.fit(X_train, y_train)
print("R^2 is:", model.score(X_test, y_test))

In [ ]:
metrics.mean_absolute_error(y_test, lr.predict(X_test))

In [ ]:
prediction = model.predict(X_test)
print("RSSE is:", (mean_squared_error(y_test, prediction)**.5))

In [ ]:
actual_values = y_test
plt.scatter(prediction, actual_values, alpha=.2, color='b') 
plt.xlabel('Predicted Price')
plt.ylabel('Actual Price')
plt.show()

#### Build a function that performs a multiple linear regression separately for each of the top 7 features and evaluates based on R^2 and MAE

In [ ]:
featurelist = ['SqFtTotLiving', 'BathFullCount', 'Bedrooms', 'SqFtGarageAttached','SqFtDeck','YrBuilt','98006'] 
def eachfeature(featurelist):
    X = None
    for feature in featurelist:
        X_train, X_test, y_train, y_test = train_test_split(np.array(
            sales_data_over_15k[feature]).reshape(-1,1), Y, random_state = 1000, test_size=.3)
        lr = linear_model.LinearRegression()
        model = lr.fit(X_train, y_train)
        print(f"R^2 for {feature} is:{model.score(X_test, y_test)}") 
        print(f"mean_absolute_error for {feature} is: {metrics.mean_absolute_error(y_test, lr.predict(X_test))}")


In [ ]:
eachfeature(featurelist)